In [2]:
from decouple import config

In [3]:
CLIENT_SECRET= config("CLIENT_SECRET")
CLIENT_ID = config("CLIENT_ID")
CLIENT_ACCESS_TOKEN = config("CLIENT_ACCESS_TOKEN")

In [30]:
import requests

def request_song_info(i):
    base_url = 'https://api.genius.com/songs/'+str(i)
    headers = {'Authorization': 'Bearer ' + CLIENT_ACCESS_TOKEN}
    
    
    response = requests.get(base_url, headers=headers)
    if response.json()["meta"]["status"] == 200:
        return [response.json()["response"]["song"]["title"],response.json()["response"]["song"]["url"]]
    else:
        return ["error"]



In [31]:
%%time 
[request_song_info(i) for i in range(3,8)]

Wall time: 1.76 s


[['Can I Live', 'https://genius.com/Jay-z-can-i-live-lyrics'],
 ['Forgive Me Father', 'https://genius.com/Fabolous-forgive-me-father-lyrics'],
 ['Down and Out', 'https://genius.com/Camron-down-and-out-lyrics'],
 ['Fly In', 'https://genius.com/Lil-wayne-fly-in-lyrics'],
 ['Lollipop (Remix)', 'https://genius.com/Lil-wayne-lollipop-remix-lyrics']]

In [12]:
r.json()["meta"]["status"]

200

In [7]:
r.json()["response"]["song"]["featured_artists"]["name"]

KeyError: 'response'

In [13]:
r.json()["response"]["song"]["url"]

'https://genius.com/Camron-killa-cam-lyrics'

In [14]:
r.json()["response"]["song"]["title"]

'Killa Cam'

In [107]:
# Progress bar
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

### Putting it all together

In [15]:
from bs4 import BeautifulSoup

In [16]:
def get_links(x,i):
    return x[i]['href']
def get_table_row(side_list_table_row):
    a = side_list_table_row.find_all("span", {"class":"metadata_unit-label"})
    label = a[0].text
    links_list = side_list_table_row.find_all("a")
    links = [get_links(links_list,i) for i in range(len(links_list))]
    return label,links
def get_table_from_page(url,title):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    side_list_table = soup.find_all("div", {"class":"metadata_unit metadata_unit--table_row"} )
    return [title, url, [get_table_row(side_list_table[i]) for i in range(0,len(side_list_table)) ]]

In [17]:
def get_song_info(i):
    r = request_song_info(i)
    if r.json()["meta"]["status"] == 200:
        title = r.json()["response"]["song"]["title"]
        url = r.json()["response"]["song"]["url"]
        return get_table_from_page(url,title)
    else:
        return ["error"+str(i)] 


    

In [22]:

get_song_info(2)

Wall time: 297 ms


['error2']

In [23]:
%%time
results = [get_song_info(i) for i in range(1,5) ]

Wall time: 2.67 s


In [24]:
results

[['Killa Cam',
  'https://genius.com/Camron-killa-cam-lyrics',
  [('Written By',
    ['https://genius.com/artists/The-heatmakerz',
     'https://genius.com/artists/Camron']),
   ('Recording Engineer', ['https://genius.com/artists/Carlisle-young']),
   ('Mixing Engineer', ['https://genius.com/artists/Carlisle-young']),
   ('Mastering Engineer', ['https://genius.com/artists/Tony-dawsey']),
   ('Label',
    ['https://genius.com/artists/Diplomat-records',
     'https://genius.com/artists/Roc-a-fella-records',
     'https://genius.com/artists/Def-jam-recordings']),
   ('Recorded At', []),
   ('Release Date', [])]],
 ['error2'],
 ['Can I Live',
  'https://genius.com/Jay-z-can-i-live-lyrics',
  [('Written By',
    ['https://genius.com/artists/Irv-gotti',
     'https://genius.com/artists/H-davis',
     'https://genius.com/artists/B-bacharach']),
   ('Mixing',
    ['https://genius.com/artists/Irv-gotti',
     'https://genius.com/artists/Joe-quinde']),
   ('Released by',
    ['https://genius.com

In [61]:
def request_artist_songs_list(i):
    page = 1
    base_url = 'https://api.genius.com/artists/'+str(i)+'/songs?per_page=50&page='+str(page)+"&sort=popularity"
    headers = {'Authorization': 'Bearer ' + CLIENT_ACCESS_TOKEN}
    
    
    response = requests.get(base_url, headers=headers)
    return response

In [67]:
%%time
r = request_artist_info(1)

Wall time: 960 ms


In [69]:
r.json()["response"]["songs"][1]["url"]

'https://genius.com/Nicki-minaj-i-am-your-leader-lyrics'

In [70]:
r.json()["response"]["next_page"]

2

In [103]:
def request_artist_id(i):

    base_url = 'https://api.genius.com/artists/'+str(i)
    headers = {'Authorization': 'Bearer ' + CLIENT_ACCESS_TOKEN}
    response = requests.get(base_url, headers=headers)
    if response.json()["meta"]["status"] == 200:
        name = response.json()["response"]["artist"]["name"]
        artist_id = response.json()["response"]["artist"]["id"]
        return (name, artist_id)
    else:
        return ("error",i)

In [104]:
r = request_artist_id(1)

In [114]:
results3 = [request_artist_id(i) for i in range(5000,10000) ]

In [115]:
results3

[('LaMaRe', 5000),
 ('Javon', 5001),
 ('Lonnie B', 5002),
 ('SupaFriendz', 5003),
 ('error', 5004),
 ('Dan Johns', 5005),
 ('Shawn Chapelle', 5006),
 ('Danny Swain', 5007),
 ('Maria', 5008),
 ('Brittany Bosco', 5009),
 ('Von Pea', 5010),
 ('Stephanie Mae', 5011),
 ('Collette', 5012),
 ('Kid Syc', 5013),
 ('Branden M. Collins', 5014),
 ('Che Grand', 5015),
 ('G. Test', 5016),
 ('Jinx (Rapper)', 5017),
 ("'Drea", 5018),
 ('Ms. Lisha', 5019),
 ('Da Notorious Prime Playaz', 5020),
 ('Mr. Ants', 5021),
 ('Amy delHierro', 5022),
 ('Faye Medina', 5023),
 ('Darc Mind', 5024),
 ('Vultcha', 5025),
 ('Darkim Be Allah', 5026),
 ('Darik the Assassin', 5027),
 ('K-Bar Allah', 5028),
 ('Black Justice', 5029),
 ('Dark Sun Riders', 5030),
 ('Brother J', 5031),
 ('Darkroom Familia (Never, Sir Dyno, Mr. Ace, Mr. Kee, Duke)', 5032),
 ('Big Supreme', 5033),
 ('Darkroom Familia', 5034),
 ('Darkside of the Force', 5035),
 ('Nocturnal', 5036),
 ('error', 5037),
 ('error', 5038),
 ('Conscious Daughters', 5039)

In [113]:
results2

[('Abeer', 1000),
 ('Natasha Bedingfield', 1001),
 ('David Milch', 1002),
 ('Robert Penn Warren', 1003),
 ('Wallace Stevens', 1004),
 ('Nadia Ackerman', 1005),
 ('Soul Khan', 1006),
 ('DJ Clue?', 1007),
 ('Pastor Troy', 1008),
 ('error', 1009),
 ('XV', 1010),
 ('error', 1011),
 ('Del The Funky Homosapien', 1012),
 ('Playboy Tre', 1013),
 ('Bishop Lamont', 1014),
 ('Chester Bennington', 1015),
 ('Swedish House Mafia', 1016),
 ('error', 1017),
 ('Lil Cuete', 1018),
 ('error', 1019),
 ('Jhené Aiko', 1020),
 ('Phonte', 1021),
 ('Evidence', 1022),
 ('Paris (Rapper)', 1023),
 ('Chubb Rock', 1024),
 ('Red Hot Lover Tone', 1025),
 ('MC Serch', 1026),
 ('error', 1027),
 ('error', 1028),
 ('Philip Larkin', 1029),
 ('error', 1030),
 ('error', 1031),
 ('Sway & King Tech', 1032),
 ('error', 1033),
 ('Blue Raspberry', 1034),
 ('\u200bk-os', 1035),
 ('error', 1036),
 ('The Cataracs', 1037),
 ('Dev', 1038),
 ('MC Hammer', 1039),
 ('Big Prime Da General', 1040),
 ('error', 1041),
 ('M.I.A.', 1042),
 ('

In [111]:
results

[("Cam'ron", 1),
 ('JAY-Z', 2),
 ('Fabolous', 3),
 ('Lil Wayne', 4),
 ('Clipse', 5),
 ('error', 6),
 ('error', 7),
 ('error', 8),
 ('error', 9),
 ('error', 10),
 ('error', 11),
 ('error', 12),
 ('Gucci Mane', 13),
 ('error', 14),
 ('error', 15),
 ('error', 16),
 ('error', 17),
 ('Big Tymers', 18),
 ('error', 19),
 ('Ghostface Killah', 20),
 ('Wu-Tang Clan', 21),
 ('The Notorious B.I.G.', 22),
 ('error', 23),
 ('error', 24),
 ('error', 25),
 ('error', 26),
 ('error', 27),
 ('error', 28),
 ('error', 29),
 ('error', 30),
 ('error', 31),
 ("Ol' Dirty Bastard", 32),
 ('error', 33),
 ('Jadakiss', 34),
 ('error', 35),
 ('error', 36),
 ('error', 37),
 ('Juvenile', 38),
 ('error', 39),
 ('error', 40),
 ('error', 41),
 ('The Game', 42),
 ('error', 43),
 ('Three 6 Mafia', 44),
 ('Eminem', 45),
 ('Snoop Dogg', 46),
 ('error', 47),
 ('error', 48),
 ('error', 49),
 ('error', 50),
 ('error', 51),
 ('error', 52),
 ('error', 53),
 ('error', 54),
 ('error', 55),
 ('Nas', 56),
 ('error', 57),
 ('Cormega'